In [ ]:
import time
import random
import networkx as nx
from networkx.algorithms import isomorphism

In [2]:
# 计算图价格：将所有顶点价格累加
def compute_price(G, v_price_dict):
    return sum(v_price_dict.get(node, {}).get("price", 0) for node in G.nodes)

# 计算图成本：将所有顶点成本累加
def compute_cost(G, v_price_dict):
    return sum(v_price_dict.get(node, {}).get("cost", 0) for node in G.nodes)

In [ ]:
def is_subisomorphic(G1, G2):
    gm = isomorphism.DiGraphMatcher(G2, G1)  # 是否 G1 是 G2 的子图
    return gm.subgraph_is_isomorphic()

def get_subisomorphic_flags(Q, subgraphs):
    flags = []
    for G in subgraphs:
        flag1 = is_subisomorphic(G, Q)
        flag2 = is_subisomorphic(Q, G)
        flags.append([flag1, flag2])
    return flags

In [ ]:
def combine_graphs(graph_list):
    combined = nx.DiGraph()
    for G in graph_list:
        combined.add_nodes_from(G.nodes(data=True))
        combined.add_edges_from(G.edges(data=True))
    return combined

In [ ]:
def subisomorphic_pricing(Q, subgraphs, v_price_dict):
    matched_exact = []
    candidates = []
    
    flags = get_subisomorphic_flags(Q, subgraphs)

    for i, (flag1, flag2) in enumerate(flags):
        if flag2:  # Q 与某图完全匹配
            price = compute_price(Q, v_price_dict)
            return [price, [subgraphs[i]]]
        elif flag1:  # Q 是某图的子图
            candidates.append(subgraphs[i])

    if not candidates:
        print("no answer")
        return [0, []]
    
    # 合并候选图，判断是否包含 Q
    G_combined = combine_graphs(candidates)
    if is_subisomorphic(Q, G_combined):
        price = compute_price(Q, v_price_dict)
        return [price, candidates]
    else:
        print("no answer")
        return [0, []]

In [ ]:
# 假设你的图所有节点都带有 name 属性
def init_price_dict(graphs, default_cost=1, default_price=2):
    v_price_dict = {}
    for G in graphs:
        for node in G.nodes:
            if node not in v_price_dict:
                v_price_dict[node] = {"cost": default_cost, "price": default_price}
    return v_price_dict

In [ ]:
def transaction_loop(Q_clu_list, subgraphs, v_price_dict, xpricing=2):
    tran_history = []
    plist_buyer = []
    re_time = []
    record_flag = []
    start_time = time.time()

    for i, Q in enumerate(Q_clu_list):
        # 定时评估
        if i in [10000, 30000, 50000, 80000, 120000]:
            e_time = time.time() - start_time
            record_flag.append(len(tran_history))
            re_time.append(e_time)
            evaluation(i / 1000, tran_history, e_time)

        # 搜索交易记录
        q_history = [h for h in tran_history if nx.is_isomorphic(Q, h[0])]
        has_history = bool(q_history)

        if not has_history:
            print(i, "query no transaction history")
            tran_info = subisomorphic_pricing(Q, subgraphs, v_price_dict)

            if tran_info[0] > 0:
                p = tran_info[0]
                matched = tran_info[1]
                if len(matched) > 1:
                    subgraphs.append(Q)
            else:
                print(i, "query no match")
                p = -1
        else:
            # 历史数据分析
            success_prices = [h[2] for h in q_history if h[1] == 1]
            fail_prices = [h[2] for h in q_history if h[1] == 0]
            sp_max = max(success_prices) if success_prices else None
            fp_min = min(fail_prices) if fail_prices else None
            p = compute_price(Q, v_price_dict)

        # 顾客心理价格
        if p > 0:
            p_buyer = Q.number_of_nodes() * 5 + random.uniform(0, 10)
            match_found = False
            for pb in plist_buyer:
                if nx.is_isomorphic(Q, pb[0]):
                    match_found = True
                    pb[1] = p_buyer  # 更新心理价格
                    break
            if not match_found:
                plist_buyer.append([Q, p_buyer])

            # 判断交易是否成功
            if p_buyer < p:
                print("交易失败")
                tran_history.append([Q, 0, p])
                if has_history and success_prices:
                    p_new = (sp_max + p) / 2
                else:
                    c = compute_cost(Q, v_price_dict)
                    if p - c > xpricing:
                        p_new = p - xpricing
                    elif p - c > xpricing / 2:
                        p_new = p - xpricing / 2
                    elif p - c > 1:
                        p_new = p - 1
                    else:
                        p_new = c + 1
            else:
                print("交易成功")
                tran_history.append([Q, 1, p])
                if has_history and fail_prices:
                    p_new = (fp_min + p) / 2
                else:
                    p_new = p + xpricing

            # 更新顶点价格
            c = compute_cost(Q, v_price_dict)
            v_price_dict = adjust_vertex_price(Q, c, p_new, v_price_dict)

    return tran_history, v_price_dict, record_flag, re_time

In [ ]:
def process_transactions_static(queries, vertex_price_list, transaction_manager, plist_buyer, evaluator, start_time):
    """
    固定定价交易处理：不使用历史信息和价格调整
    """
    evaluation_intervals = {10000, 30000, 50000, 80000, 120000, 150000}
    results = {}

    for i, Q in enumerate(queries, start=1):
        # Step 1: 周期性评估
        if i in evaluation_intervals:
            e_time = time.time() - start_time
            success_ratio, avg_regret, avg_price_deviation, avg_sdiff, avg_diff, avg_per_diff, customer_avg_per_diff = evaluator.evaluate_transactions(transaction_manager.get_history())
            results = {
                "success_ratio": success_ratio,
                "avg_regret": avg_regret,
                "avg_price_deviation": avg_price_deviation,
                "avg_sdiff": avg_sdiff,
                "avg_diff": avg_diff,
                "avg_per_diff": avg_per_diff,
                "customer_avg_per_diff": customer_avg_per_diff,
                "time": e_time
            }
            evaluator.save_evaluation_results(i, "TEST_static_evaluation_results.txt", results)

        expected_price = evaluator.generate_expected_price(Q)
        price = compute_pricing(Q, vertex_price_list)
        print('exp:', expected_price, 'price:', price)

        if price == float('inf'):
            print("Error: 有顶点价格为 inf，跳过该查询 Q")
            continue

        # Step 2: 更新顾客心理价格列表
        found = False
        for p_b in plist_buyer:
            if gequal(Q, p_b[0]):
                found = True
                p_b[1] = expected_price
        if not found:
            plist_buyer.append([Q, expected_price])

        success = expected_price >= price
        print("交易" + ("成功" if success else "失败") + f" - Q: {Q}, 价格: {price}, 预期价格: {expected_price}")

        transaction_manager.add_transaction(Q, price, success)